In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

# 定义每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

# 参数概要
def variable_summaries(var)：
    with tf.name_scope('summaries'):
        mean =  tf.reduce_mean(var)
        tf.summary.scalar("mean",mean) # 平均值
        with tf.name_scope('stddev'):
            stddev =  tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            
        tf.summary.scalar("stddev",stddev) # 标准差
        tf.summary.scalar("max",tf.reduce_max(var)) # 最大值
        tf.summary.scalar("min",tf.reduce_min(var)) # 最小值
        tf.summary.histogram("histogram",var) # 直方图

# 初始化权值的函数
def weight_variable(shape):
    # 生成一个截断的正态分布
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1)) 

# 初始化偏值
def bias_variable(shape):
    return tf.Variable(tf.constant(0.1,shape=shape)) 

# 卷积层（2维）
def mConv2d(input,filter):
    # input: 4-D tensor[`batch`, `img_height`,`img_width`, `img_channels`]
    # filter （卷积核）: A 4-D tensor.`[filter_height, filter_width, in_channels, out_channels]`
    # strides (步长): 1-D tensor。固定：strides[0] = strides[3] = 1. strides[1]代表x方向步长，strides[2]代表y方向步长 
    return tf.nn.conv2d(input,filter,strides=[1,1,1,1],padding='SAME') 

# 池化层
def max_pool_2x2(input):
    # ksize （窗口大小） [1,x,y,1]   固定：ksize[0] = ksize[3] = 1
     return tf.nn.max_pool(input,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')  # SAME 、 VALID(不超出平面外部)

# 命名空间（input）
with tf.name_scope("input"):
    # # 根据样本样式定义创建占位符（1列）
    x = tf.placeholder(tf.float32,[None,784], name='x-input') # 每张图的像素点信息： 28*28 = 784
    y = tf.placeholder(tf.float32,[None,10], name='y-input')  # 十个数字

    with tf.name_scope("x_image"):
        # 改变 x 的格式转为4D的向量 [batch,h,w,channel].(复原为原来图片的样子)
        # 图片二维数据为28*28 ，一维数据格式为 784 长度的数组
        x_image = tf.reshape(x,[-1,28,28,1], name='x_image')

# ---------------------------------------------------------------
# 定义卷积结构
# ---------------------------------------------------------------

with tf.name_scope("Conv1"):                             
    # 初始化第一个卷基层
    with tf.name_scope("W_conv1"):
        W1 = weight_variable([5,5,1,32],  name='W_conv1')  # 5*5的采样窗口，32个卷积核从1一个平面抽取特征 。
    with tf.name_scope("b_conv1"):
        b1 = bias_variable([32],  name='b_conv1')
        
    with tf.name_scope("h_convl"):   
        # 把 x_image 和权值向量进行卷积，在加上偏置值，然后应用于relu激活函数 
        h_convl = tf.nn.relu(mConv2d(x_image,W1) + b1)
    with tf.name_scope("h_pool1"):   
        # 进行池化
        h_pool1 =  max_pool_2x2(h_convl)
        
        
with tf.name_scope("Conv2"):                             
    # 初始化第二个卷基层
    with tf.name_scope("W_conv2"):
        W2 = weight_variable([5,5,32,64],  name='W_conv2')  # 5*5的采样窗口，64个卷积核从32个平面抽取特征，得到64个特征平面
    with tf.name_scope("b_conv2"):
        b2 = bias_variable([64],  name='b_conv2')
        
    with tf.name_scope("h_conv2"):   
        # 把 x_image 和权值向量进行卷积，在加上偏置值，然后应用于relu激活函数 
        h_conv2 = tf.nn.relu(mConv2d(h_pool1,W2) + b2)
    with tf.name_scope("h_pool2"):   
        # 进行池化
        h_pool2 =  max_pool_2x2(h_conv2)

# ---------------------------------------------------------------
# 分析 
# ---------------------------------------------------------------
# 1. 28*28 的图片第一次卷积后还是 28*28 （因为卷积核步长为 1*1）
# 2. 第一次池化后，则变为 14*14 （因为池化层步长为 2*2）
# 3. 则接下来，第二次对 14*14 进行卷积后还是 14*14 
# 4. 第二次池化后，则变为 7*7 

# ---------------------------------------------------------------
# 定义神经网络结构
# ---------------------------------------------------------------

# 初始化一个全连接层的权值
W_fc1 = weight_variable([7*7*64,1024])  # 一共得到64个特征平面，每个平面为 7*7 。1024个神经元
b_fc1 = bias_variable([1024])

# 把池化层2的输出扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
# 求第一个全连接层的输出
h_fc1 =  tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

# keep_prob 用来表示神经元的输出概率
keep_prob = tf.placeholder(tf.float32) 
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

# 初始化第二个全连接层的权值
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])

prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

# 交叉熵
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = prediction))

# 使用AdamOptimizer进行优化
train =  tf.train.AdamOptimizer(1e-4).minimize(loss)

# 结果存放在一个布尔型的列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))# argmax 返回一维张量中最大值的所以在位置
# 求准确率
# cast 把布尔型列表转换为float32， 如[true.true.false] =》 [1,1,0] ，那么准确率的值即为66.6%
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess: 
    # 初始化全部变量
    sess.run(tf.global_variables_initializer())
    # 训练21个周期
    for epoch in range(21):
        # n_batch ： 一共有多少个批次
        for batch in range(n_batch):
            # 保存batch_size张图片的数据与标签
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        
        # 用测试集的图片及标签求得准确率
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter "+ str(epoch) + ", Texting acc " + str(acc))
